<a href="https://colab.research.google.com/github/LiangHao09/Detecctor-de-amarillismo/blob/main/Detecctor_de_amarillismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>